In [19]:
import json
import codecs
import collections
import pandas as pd
import networkx as nx

from gensim import corpora, models

from IPython.display import display, HTML

In [26]:
G = nx.read_gexf("datasets/influences.gexf")
communities = json.load(codecs.open("datasets/communities/louvain.json","r", "utf-8"))

In [ ]:
def get_sentences(doc_name):
  sentences = []
  sentences = codecs.open("datasets/sentences/%s.txt" % doc_name.replace("/", "-"),"r", "utf-8").read().split("\n")
  sentences = [ s.split() for s in sentences ]
  
  return sentences

def community_bow(community):
  sentences = reduce(lambda a,b: a + b, map(get_sentences, community))
  
  bigram = models.phrases.Phrases(sentences)
  words = reduce(lambda a,b: a+b, bigram[sentences])
  counter = collections.Counter(words)

  bow = counter.most_common(100)

  return bow

bags = dict()

for k in sorted([ int(k) for k in communities.keys()]):
  print "processing community k=%s" % k
  
  bags[str(k)] = community_bow(communities[str(k)])

In [31]:
def print_groups(communities):
    html = "<table>"

    for c, words in communities:
        html += "<tr><td style=\"width: 100px; text-align: center; \"><h3>group %s</h3></td><td><ul>" % (c)
        html += "\n".join([ "<li>%s (%s)</li>" % (word, score) for word, score in words ])
        html += "</ul></td></tr>"
    
    html += "</table>"

    display(HTML(html))

print_groups(map(lambda k: (k, bags[str(k)][0:10]), sorted([ int(k) for k in communities.keys()])))

group 0,ibn (1087) muhammad (878) islam (588) ali (543) islamic (540) p (503) also (439) 2 (406) muslim (384) book (372)
group 1,p (2704) de (2539) also (2094) philosophy (2093) first (1881) works (1850) 2 (1807) work (1795) john (1593) life (1582)
group 2,p (2946) pp (2068) new_york (1719) also (1607) 2 (1605) press (1508) first (1506) their (1428) work (1417) de (1396)
group 3,philosophy (2519) work (2087) new_york (1723) 2 (1663) press (1659) p (1625) also (1623) life (1497) de (1389) history (1302)
group 4,philosophy (797) de (676) p (657) work (614) 2 (585) also (579) press (469) new_york (460) life (455) university (453)
group 5,p (916) first (642) 2 (626) also (616) greek (614) press (590) after (577) life (558) de (546) pp (541)
group 6,philosophy (2325) press (1257) university (1204) theory (1131) 2 (1116) work (1103) p (1077) also (1036) 3 (829) new_york (821)
group 7,p (1151) also (771) de (709) 2 (671) after (623) first (617) their (571) new (539) new_york (532) press (515)
group 8,social (417) new_york (361) press (345) education (342) philosophy (314) 2 (295) university (285) work (285) theory (276) their (272)
group 9,church (329) luther (288) erasmus (221) de (208) john (205) life (190) thomas (170) works (162) p (158) their (154)
group 10,philosophy (1478) p (1348) also (1324) 2 (1273) work (1034) first (1021) life (1016) works (979) 3 (959) press (924)
